In [34]:
from mylib.models.misc import set_gpu_usage

set_gpu_usage()

from mylib.dataloader.dataset import ClfSegDataset, get_balanced_loader, get_loader
from mylib.models import densesharp, metrics, losses,densenet

from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

batch_sizes=[3, 5, 8, 8]
crop_size=[32, 32, 32]
random_move=3
learning_rate=1.e-4
segmentation_task_ratio=0.2
weight_decay=0.
save_folder='./TrainedModel'
epochs=100


In [3]:
batch_size = sum(batch_sizes)

train_dataset = ClfSegDataset(crop_size=crop_size, subset=[ 1, 2, 3], move=random_move,
                                  define_label=lambda l: [l[0] ,l[1]])

val_dataset = ClfSegDataset(crop_size=crop_size, subset=[0], move=None,
                                define_label=lambda l: [l[0] , l[1]])

train_loader = get_balanced_loader(train_dataset, batch_sizes=batch_sizes)
val_loader = get_loader(val_dataset, batch_size=batch_size)

In [4]:
model = densesharp.get_compiled(output_size=2,
                                    optimizer=Adam(lr=learning_rate),
                                    loss={"clf": 'categorical_crossentropy',
                                          "seg": losses.DiceLoss()},
                                    metrics={'clf': ['accuracy', metrics.precision, metrics.recall, metrics.fmeasure,
                                                     metrics.invasion_acc, metrics.invasion_fmeasure,
                                                     metrics.invasion_precision, metrics.invasion_recall,
                                                     metrics.ia_acc, metrics.ia_fmeasure,
                                                     metrics.ia_precision, metrics.ia_recall],
                                             'seg': [metrics.precision, metrics.recall, metrics.fmeasure]},
                                    loss_weights={"clf": 1., "seg": segmentation_task_ratio},
                                    weight_decay=weight_decay)

Model hyper-parameters: {'activation': <function <lambda> at 0x00000203810211F8>, 'bn_scale': True, 'weight_decay': 0.0, 'kernel_initializer': 'he_uniform', 'first_scale': <function <lambda> at 0x00000203A3F55EE8>, 'dhw': [32, 32, 32], 'k': 16, 'bottleneck': 4, 'compression': 2, 'first_layer': 32, 'down_structure': [4, 4, 4], 'output_size': 2, 'dropout_rate': None}
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 32, 1 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 32, 32, 32, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv3d_1 (Conv3D)    

In [18]:
densenet_model = densenet.get_compiled()

Model hyper-parameters: {'activation': <function <lambda> at 0x00000203F848B0D8>, 'bn_scale': True, 'weight_decay': 0.0001, 'kernel_initializer': 'he_uniform', 'first_scale': <function <lambda> at 0x00000203EC74AC18>, 'dhw': [32, 32, 32], 'k': 16, 'bottleneck': 4, 'compression': 2, 'first_layer': 32, 'down_structure': [4, 4, 4], 'output_size': 2}
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 32, 1 0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 32, 32, 32, 1 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv3d_28 (Conv3D)              (None, 3

In [19]:
checkpointer = ModelCheckpoint(filepath='tmp/%s/weights.{epoch:02d}.h5' % save_folder, verbose=1,
                                   period=1, save_weights_only=False)
best_keeper = ModelCheckpoint(filepath='tmp/%s/best.h5' % save_folder, verbose=1, save_weights_only=False,
                                  monitor='val_clf_acc', save_best_only=True, period=1, mode='max')
csv_logger = CSVLogger('tmp/%s/training.csv' % save_folder)
tensorboard = TensorBoard(log_dir='tmp/%s/logs/' % save_folder)
early_stopping = EarlyStopping(monitor='val_clf_acc', min_delta=0, mode='max',
                                   patience=30, verbose=1)
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.334, patience=10,
                                   verbose=1, mode='min', epsilon=1.e-5, cooldown=2, min_lr=0)


F:\anaconda3\envs\kr\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [48]:
net_history=densenet_model.fit_generator(generator=train_loader, steps_per_epoch=len(train_dataset), max_queue_size=500, workers=1,
                        validation_data=val_loader, epochs=10, validation_steps=len(val_dataset),verbose=0,
                                         callbacks=[checkpointer, tensorboard])
                        #callbacks=[checkpointer, early_stopping, best_keeper, lr_reducer, csv_logger, tensorboard])



Epoch 00001: saving model to tmp/./TrainedModel/weights.01.h5

Epoch 00002: saving model to tmp/./TrainedModel/weights.02.h5

Epoch 00003: saving model to tmp/./TrainedModel/weights.03.h5

Epoch 00004: saving model to tmp/./TrainedModel/weights.04.h5

Epoch 00005: saving model to tmp/./TrainedModel/weights.05.h5

Epoch 00006: saving model to tmp/./TrainedModel/weights.06.h5

Epoch 00007: saving model to tmp/./TrainedModel/weights.07.h5

Epoch 00008: saving model to tmp/./TrainedModel/weights.08.h5

Epoch 00009: saving model to tmp/./TrainedModel/weights.09.h5

Epoch 00010: saving model to tmp/./TrainedModel/weights.10.h5


In [ ]:
net_history2=densenet_model.fit(x_train,y_train steps_per_epoch=len(train_dataset), max_queue_size=500, workers=1,
                        validation_data=val_loader, epochs=10, validation_steps=len(val_dataset),verbose=0,
                                         callbacks=[checkpointer, tensorboard])
                        #callbacks=[checkpointer, early_stopping, best_keeper, lr_

In [57]:
print(net_history.history.keys())
      

dict_keys([])


In [56]:
net_history=densenet_model.fit_generator(generator=train_loader, steps_per_epoch=len(train_dataset), max_queue_size=500, workers=1,
                        validation_data=val_loader, epochs=10, validation_steps=len(val_dataset),verbose=0,callbacks=[checkpointer, early_stopping, best_keeper, lr_reducer, csv_logger, tensorboard])


Epoch 00001: saving model to tmp/./TrainedModel/weights.01.h5

Epoch 00002: saving model to tmp/./TrainedModel/weights.02.h5

Epoch 00003: saving model to tmp/./TrainedModel/weights.03.h5

Epoch 00004: saving model to tmp/./TrainedModel/weights.04.h5

Epoch 00005: saving model to tmp/./TrainedModel/weights.05.h5

Epoch 00006: saving model to tmp/./TrainedModel/weights.06.h5

Epoch 00007: saving model to tmp/./TrainedModel/weights.07.h5

Epoch 00008: saving model to tmp/./TrainedModel/weights.08.h5

Epoch 00009: saving model to tmp/./TrainedModel/weights.09.h5

Epoch 00010: saving model to tmp/./TrainedModel/weights.10.h5


In [6]:
history=model.fit_generator(generator=train_loader, steps_per_epoch=len(train_dataset), max_queue_size=500, workers=1,
                        validation_data=val_loader, epochs=epochs, validation_steps=len(val_dataset),verbose=0,
                        callbacks=[checkpointer, early_stopping, best_keeper, lr_reducer, csv_logger, tensorboard])
   

Size 117

Epoch 00001: saving model to tmp/./TrainedModel/weights.01.h5


F:\anaconda3\envs\kr\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_clf_acc` which is not available. Available metrics are: 
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
F:\anaconda3\envs\kr\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_clf_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
F:\anaconda3\envs\kr\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning



Epoch 00002: saving model to tmp/./TrainedModel/weights.02.h5

Epoch 00003: saving model to tmp/./TrainedModel/weights.03.h5

Epoch 00004: saving model to tmp/./TrainedModel/weights.04.h5

Epoch 00005: saving model to tmp/./TrainedModel/weights.05.h5

Epoch 00006: saving model to tmp/./TrainedModel/weights.06.h5

Epoch 00007: saving model to tmp/./TrainedModel/weights.07.h5

Epoch 00008: saving model to tmp/./TrainedModel/weights.08.h5

Epoch 00009: saving model to tmp/./TrainedModel/weights.09.h5

Epoch 00010: saving model to tmp/./TrainedModel/weights.10.h5

Epoch 00011: saving model to tmp/./TrainedModel/weights.11.h5

Epoch 00012: saving model to tmp/./TrainedModel/weights.12.h5

Epoch 00013: saving model to tmp/./TrainedModel/weights.13.h5

Epoch 00014: saving model to tmp/./TrainedModel/weights.14.h5

Epoch 00015: saving model to tmp/./TrainedModel/weights.15.h5

Epoch 00016: saving model to tmp/./TrainedModel/weights.16.h5

Epoch 00017: saving model to tmp/./TrainedModel/weight

In [32]:
print(net_history.history())

TypeError: 'dict' object is not callable

In [31]:
print(net_history.history.keys())

dict_keys([])


In [ ]:

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')


In [12]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'], label='acc')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

KeyError: 'acc'

In [8]:
from mylib.dataloader import dataset1
import numpy as np

clf_seg_dataset = dataset1.ClfSegDataset(crop_size=32, subset=[1,2,3])
train_num=len(clf_seg_dataset)
x_train= np.ones((train_num,32,32,32,1))
y_train= np.ones((train_num,1))
print(x_train.shape)
i=0
while i<train_num:
    x_train[i],(y_train[i],seg)=clf_seg_dataset[i]
    i+=1

from keras.utils import to_categorical
y_train=to_categorical(y_train,2)



(354, 32, 32, 32, 1)


In [1]:
y_pre=model.predict(x_train)

NameError: name 'model' is not defined

In [10]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

KeyError: 'accuracy'